In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import torch.optim as optim

# import numpy as np
import pandas as pd
import ast
import math
import time

from binary_utils import bin_decoder, bin_encoder, int_to_binary_str, binary_str_to_int
from models.rnn.encoder_decoder_gru import EncoderRNN, DecoderRNN
from models.rnn.combined_networks import train
from utils import showPlot, timeSince, asMinutes

%load_ext autoreload
%autoreload 2

# Training Loops

In [3]:
input_seq = torch.Tensor([0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0]) # -> binary encoded integer sequence
target_seq = torch.Tensor([0, 2, 5, 6]) # -> character encoded function

training_pairs = [[input_seq, target_seq]]

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
def training(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    # training_pairs = [tensorsFromPair(random.choice(pairs))
    #                   for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [6]:
hidden_size = len(input_seq)
encoder = EncoderRNN(len(input_seq), hidden_size).to(device)
decoder = DecoderRNN(hidden_size, len(target_seq)).to(device)

training(encoder, decoder, 1, print_every=5000)

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)